<a href="https://colab.research.google.com/github/shinchan75034/testrep/blob/master/tf20_dist_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install tensorflow-gpu==2.0.0-rc1

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5616540783915904173
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8964607837846894019
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5305775699766031626
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11330115994
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8054168635746750968
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]
2.0.0-rc1


In [3]:
! pip install tf-explain

In [0]:
import tf_explain
import numpy as np
import time

In [0]:
INPUT_SHAPE = (28, 28, 1)
NUM_CLASSES = 10

AVAILABLE_DATASETS = {
    'mnist': tf.keras.datasets.mnist,
    'fashion_mnist': tf.keras.datasets.fashion_mnist,
}
DATASET_NAME = 'fashion_mnist' 

In [0]:
# Load dataset
dataset = AVAILABLE_DATASETS[DATASET_NAME]
(train_images, train_labels), (test_images, test_labels) = dataset.load_data()

In [0]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

# Convert from (28, 28) images to (28, 28, 1) '...' is equivalent to ':' which means everything. now train images are shape 60k, 28,28,1
train_images = train_images[..., tf.newaxis]
test_images = test_images[..., tf.newaxis]

In [0]:
idx = np.random.choice(len(train_images), 10000, replace = False)
val_images = train_images[idx]
val_labels = train_labels[idx]
train_images = np.delete(train_images, idx, axis=0)
train_labels = np.delete(train_labels, idx, axis=0)

In [0]:
# Convert numpy to tf.data.Dataset
# This is from https://www.tensorflow.org/beta/tutorials/load_data/numpy

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))

In [10]:

strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [0]:
BUFFER_SIZE = 100000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

my_train_dataset = train_dataset.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
my_eval_dataset = test_dataset.map(scale).batch(BATCH_SIZE)
my_val_dataset = val_dataset.map(scale).shuffle(BUFFER_SIZE).batch(1)

In [13]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [0]:
 # Define the checkpoint directory to store the checkpoints

import os
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")


# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
      if epoch < 3:
          return 1e-3
      elif epoch >= 3 and epoch < 7:
          return 1e-4
      else:
          return 1e-5

In [0]:
# Select a subset of the validation data to examine
# Here, we choose 5 elements with label "0" == [1, 0, 0, .., 0]
validation_class_zero = (np.array([
    el for el, label in zip(tf.cast(test_images, tf.float32), tf.keras.utils.to_categorical(test_labels, num_classes=NUM_CLASSES))
    if np.all(label == np.array([1] + [0] * 9))
][0:5]), None)
# Select a subset of the validation data to examine
# Here, we choose 5 elements with label "4" == [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
validation_class_fours = (np.array([
    el for el, label in zip(tf.cast(test_images, tf.float32), tf.keras.utils.to_categorical(test_labels, num_classes=NUM_CLASSES))
    if np.all(label == np.array([0] * 4 + [1] + [0] * 5))
][0:5]), None)

In [16]:
# One hot encore labels 0, 1, .., 9 to [0, 0, .., 1, 0, 0]
#train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=NUM_CLASSES)
#test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=NUM_CLASSES)

callbacks = [

    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_best_only=True, monitor='val_acc', mode='max',
                                       save_weights_only=False),
    tf.keras.callbacks.LearningRateScheduler(decay)

]


model.fit(my_train_dataset, validation_data=my_val_dataset,
        steps_per_epoch=5,
        validation_steps=5, epochs=5, callbacks=callbacks)


Train for 5 steps, validate for 5 steps
Epoch 1/5
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
1/5 [=====>........................] - ETA: 18s - loss: 2.3169 - accuracy: 0.0469INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
5/5 [========================